In [1]:
train_data = []
with open('./pos.txt','r',encoding = 'utf-8') as f:
    for ele in f:
        if "\\" in ele:
            pass
        else:
            train_data.append(ele)

In [3]:
train_target= []
for i in range(len(train_data)):
    train_target.append('pos')

In [4]:
train_neg= []
with open('./neg.txt','r',encoding = 'utf-8') as f:
    for ele in f:
        if "\\" in ele:
            pass
        else:
            train_neg.append(ele)

In [5]:
for i in range(len(train_neg)):
    train_target.append('neg')

In [6]:
train_neu= []
with open('./neu.txt','r',encoding = 'utf-8') as f:
    for ele in f:
        if "\\" in ele:
            pass
        else:
            train_neu.append(ele)

In [7]:
for i in range(len(train_neu)):
    train_target.append('neu')

In [8]:
train_data = train_data+train_neg+train_neu

In [11]:
import random
combine = list(zip(train_data,train_target))
random.shuffle(combine)
new_train_data,new_train_target = zip(*combine)

In [42]:
len(new_train_data)

100335

In [53]:
final_train_data = new_train_data[0:80000]
final_train_target = new_train_target[0:80000]

final_test_data = new_train_data[80001:100334]
final_test_target = new_train_target[80001:100334]

In [54]:
from sklearn.feature_extraction.text import CountVectorizer

In [55]:
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.util import ngrams

In [56]:
snow_stemmer = SnowballStemmer(language='english')

In [57]:
def custom_tokenizer(text, use_stem=True, stemmer=snow_stemmer, use_pos=False, 
                     use_only_adj=False, use_bigrams=False, use_bigrams_only=False):
    # Separate words
    words = word_tokenize(text)
    # PoS tagging words
    if use_pos:
        pos_tags = nltk.pos_tag(words)
    else:
        pos_tags = zip(words, [''] * len(words))
    
    tokens = []
    # Special treatment for bigrams
    if use_bigrams:
        tokens += list(ngrams(words, n=2))
        if use_bigrams_only:
            return tokens
        else:
            tokens += [(x,) for x in words]
        return tokens
    
    for word, tag in pos_tags:
        res_word = word
        use_word = True
        # Convert to stem
        if use_stem:
            res_word = stemmer.stem(res_word)
        # Use POS tag with the word
        if use_pos and not use_only_adj:
            res_word += '_' + tag
        # Only use adv and adj
        if use_only_adj and not (tag[:2] == 'JJ' or tag[:2] == 'RB'):
            use_word = False
        # Append the word to the tokenizer
        if use_word:
            tokens.append(res_word)
    return tokens

def text_stems_tok(text):
    return custom_tokenizerC
def pos_tok(text):
    return custom_tokenizer(text, use_stem=False, use_pos=True)
def pos_stems_tok(text):
    return custom_tokenizer(text, use_stem=True, use_pos=True)
def adj_tok(text):
    return custom_tokenizer(text, use_stem=False, use_pos=True, use_only_adj=True)
def adj_stems_tok(text):
    return custom_tokenizer(text, use_stem=True, use_pos=True, use_only_adj=True)
def unigrams(text):
    return word_tokenize(text)
def uni_bigrams(text):
    return custom_tokenizer(text, use_bigrams=True)
def bigrams(text):
    return custom_tokenizer(text, use_bigrams=True, use_bigrams_only=True)
def uni_bigrams_stems(text):
    tokens = uni_bigrams(text)
    res_tokens = []
    for t in tokens:
        res_tokens.append(tuple([snow_stemmer.stem(x) for x in t]))
    return res_tokens

In [58]:
count_vect = CountVectorizer(tokenizer=unigrams)
X_train_counts = count_vect.fit_transform(final_train_data)
X_train_counts.shape

(80000, 42581)

In [59]:
X_train_counts

<80000x42581 sparse matrix of type '<class 'numpy.int64'>'
	with 628867 stored elements in Compressed Sparse Row format>

### TF-IDF

In [60]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=True).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(80000, 42581)

### SVM

In [61]:
from sklearn.linear_model import SGDClassifier
text_clf = SGDClassifier()
text_clf.fit(X_train_tf, final_train_target)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [62]:
from sklearn import svm
svc_model = svm.SVC(gamma=0.001, C=100., kernel='linear')
svc_clf = Pipeline([('vect', CountVectorizer(tokenizer=uni_bigrams_stems)),
                     ('tfidf', TfidfTransformer(use_idf=True)),
                     ('clf', svc_model)
                    ])

svc_clf = svc_clf.fit(final_train_data, final_train_target)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: generator 'ngrams' raised StopIteration
  


In [63]:
import numpy as np
predicted = svc_clf.predict(final_test_data)
np.mean(predicted == final_test_target)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: generator 'ngrams' raised StopIteration
  


0.9110805095165494

In [64]:
from sklearn import svm
svc_model = svm.SVC(gamma=0.001, C=100., kernel='rbf')
svc_clf = Pipeline([('vect', CountVectorizer(tokenizer=uni_bigrams_stems)),
                     ('tfidf', TfidfTransformer(use_idf=True)),
                     ('clf', svc_model)
                    ])

svc_clf = svc_clf.fit(final_train_data, final_train_target)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: generator 'ngrams' raised StopIteration
  


In [65]:
import numpy as np
predicted = svc_clf.predict(final_test_data)
np.mean(predicted == final_test_target)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: generator 'ngrams' raised StopIteration
  


0.88250627059459996

In [66]:
from sklearn import svm
svc_model = svm.SVC(gamma=0.001, C=100., kernel='sigmoid')
svc_clf = Pipeline([('vect', CountVectorizer(tokenizer=uni_bigrams_stems)),
                     ('tfidf', TfidfTransformer(use_idf=True)),
                     ('clf', svc_model)
                    ])

svc_clf = svc_clf.fit(final_train_data, final_train_target)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: generator 'ngrams' raised StopIteration
  


In [67]:
import numpy as np
predicted = svc_clf.predict(final_test_data)
np.mean(predicted == final_test_target)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: generator 'ngrams' raised StopIteration
  


0.86111247725372542

### Sklearn pipeline

In [68]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier

text_clf = Pipeline([('vect', CountVectorizer(tokenizer=uni_bigrams_stems)),
                     ('tfidf', TfidfTransformer(use_idf=True)),
                     ('clf', SGDClassifier())
                    ])
                     

In [69]:
text_clf = text_clf.fit(final_train_data, final_train_target)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: generator 'ngrams' raised StopIteration
  


In [70]:
import numpy as np
predicted = text_clf.predict(final_test_data)
np.mean(predicted == final_test_target)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: generator 'ngrams' raised StopIteration
  


0.86691585107952585

In [71]:
#export the model
from sklearn.externals import joblib
joblib.dump(text_clf, 'svm.pkl', compress=9)

['svm.pkl']

### Logistic Regression

In [72]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
new_text_pipe = Pipeline([
    ('bow', CountVectorizer(ngram_range=(1,2), min_df=2)),
    #('tf-idf', TfidfTransformer()),
    ('lr', LogisticRegression())
])

In [73]:
lr_clf = new_text_pipe.fit(final_train_data, final_train_target)

In [74]:
import numpy as np
predicted = lr_clf.predict(final_test_data)
np.mean(predicted == final_test_target)

0.91265430580829199

In [75]:
#export the model
from sklearn.externals import joblib
joblib.dump(lr_clf, 'logistic.pkl', compress=9)

['logistic.pkl']

### ANN

In [175]:
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
mlp = Pipeline([
    ('bow', CountVectorizer(ngram_range=(1,2), min_df=2)),
    #('tf-idf', TfidfTransformer()),
    ('mlp', MLPClassifier())
])

In [176]:
mlp_ANN = mlp.fit(final_train_data, final_train_target)

In [177]:
import numpy as np
predicted = mlp_ANN.predict(final_test_data)
np.mean(predicted == final_test_target)

0.75996857198978585

### KNN

In [76]:
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
knn = Pipeline([
    ('bow', CountVectorizer(ngram_range=(1,2), min_df=2)),
    #('tf-idf', TfidfTransformer()),
    ('knn', KNeighborsClassifier())
])

In [77]:
knn_clf = knn.fit(final_train_data, final_train_target)

In [78]:
import numpy as np
predicted = knn_clf.predict(final_test_data)
np.mean(predicted == final_test_target)

0.84325972556927165

In [79]:
#export the model
from sklearn.externals import joblib
joblib.dump(knn_clf, 'KNN.pkl', compress=9)

['KNN.pkl']

### Naive Bayes

In [160]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
NB = Pipeline([
    ('bow', CountVectorizer(ngram_range=(1,2), min_df=2)),
    #('tf-idf', TfidfTransformer()),
    ('NB', MultinomialNB())
])

In [162]:
nb = NB.fit(final_train_data, final_train_target)

In [163]:
import numpy as np
predicted = nb.predict(final_test_data)
np.mean(predicted == final_test_target)

0.80632488705558825

### AdaBoostTree Tree

In [172]:
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
adaboost = Pipeline([
    ('bow', CountVectorizer(ngram_range=(1,2), min_df=2)),
    #('tf-idf', TfidfTransformer()),
    ('adaboost', AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=50),n_estimators=120, learning_rate=0.1))
])

In [173]:
ad = adaboost.fit(final_train_data, final_train_target)

In [174]:
import numpy as np
predicted = ad.predict(final_test_data)
np.mean(predicted == final_test_target)

0.78570025535258303